In [1]:
import httplib2
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from parsel import Selector
from statistics import mean
import time
import requests
from datetime import datetime
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
http = httplib2.Http()
import mysql.connector

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from parsel import Selector
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

# configure webdriver
options = Options()
options.headless = True  # hide GUI
options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
options.add_argument("start-maximized")  # ensure window is full-screen
# configure chrome browser to not load images and javascript
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option(
    "prefs", {"profile.managed_default_content_settings.images": 2}
)

In [3]:
##create database
justwatch_tv_db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="4156"
)

mycursor = justwatch_tv_db.cursor(buffered=True)
mycursor.execute("CREATE DATABASE IF NOT EXISTS justwatch_tv_us_db")

In [4]:
mycursor.execute("use justwatch_tv_us_db")

In [5]:
mycursor.execute("""CREATE TABLE IF NOT EXISTS tv_links(title_link TEXT, is_scraped TINYINT)""")

In [6]:
##connect to db
from sqlalchemy import create_engine

# Credentials to database connection
hostname="localhost"
dbname="justwatch_tv_us_db"
uname="root"
pwd="4156"


# Create SQLAlchemy engine to connect to MySQL Database
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
				.format(host=hostname, db=dbname, user=uname, pw=pwd))

In [7]:
base_url = "https://www.justwatch.com"

In [35]:
# Python3 Program to Create list 
# with integers within given range 
def get_search_links(year_from, year_to): 
    def createYears(year_from, year_to):
        return [item for item in range(year_from, year_to)]
        
    genres = ['act','cmy',  'doc', 'fnt', 'hrr', 'msc', 'rms', 'spt', 'wsn', 'ani', 'crm', 'drm', 'hst', 'fml', 'trl', 'scf', 'war', 'rly']
    years = createYears(year_from, year_to)


    search_links = []
    for i in genres:
        for j in years:
            link = f"https://www.justwatch.com/us/tv-shows?genres={i}&release_year_from={j}&release_year_until={j}"
            search_links.append(link)
    
    return search_links

In [38]:
def get_title_links(link):
    titleLinks = []
    print(link)
    chrome_options = Options()  
    chromedriver = webdriver.Chrome(ChromeDriverManager().install())
    with chromedriver as browser:
        browser.get(link)
        time.sleep(5)

        last_height = browser.execute_script("return document.body.scrollHeight")
        SCROLL_PAUSE_TIME = 5
        while True:
                # Scroll down to bottom
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                # Wait to load page
                time.sleep(SCROLL_PAUSE_TIME)

                # Calculate new scroll height and compare with last scroll height
                new_height = browser.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height
                
        page_source =browser.page_source
        soup=BeautifulSoup(page_source,'html.parser')
        productlist = soup.find("div",{"listlayout":"Grid"})
        titles = productlist.find_all("div", {"class":"title-list-grid__item"})
        for title in titles:
                    title_link = title.find("a").get('href')
                    title_link = base_url+title_link
                    is_scraped = 0
                    if title_link not in titleLinks:
                        titleLinks.append(title_link)
                    
                    mycursor.execute("""INSERT INTO tv_links VALUES(%s, %s)""", (title_link, is_scraped))
                    justwatch_tv_db.commit()
                    
        
        return titleLinks
                    
                        
                        

In [50]:
mycursor.execute("""SELECT * FROM tv_links""")
results = mycursor.fetchall()
len(results)

18168

In [57]:
df_links = pd.read_sql('SELECT * FROM tv_links WHERE is_scraped = 0', con=justwatch_tv_db)
title_urls = []
for i in df_links['title_link']:
    title_urls.append(i)

c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [20]:
def get_stream_options(soup_object):
        stream_list = []
        price_comparison = soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_stream = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--stream price-comparison__grid__row--block"})
        for comp in price_comparison_stream:
            comp = comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for comp in comp:
                service = comp.find("img")
                service = service['title']
                pricing = comp.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                stream_options = {service:pricing}
                stream_list.append(stream_options)
        return stream_list


def get_rent_options(soup_object):
        rent_list = []
        price_comparison = soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_rent = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--rent price-comparison__grid__row--block"})
        for comp in price_comparison_rent:
            comp =  comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for element in comp:
                service = element.find("img")
                service = service['title']
                pricing = element.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                rent_options = {service:pricing}
                rent_list.append(rent_options)
        
        return rent_list
    
def get_buy_options(soup_object):
        buy_list = []
        price_comparison = soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_buy = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--buy price-comparison__grid__row--block"})
        for comp in price_comparison_buy:
            comp = comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for element in comp:
                service = element.find("img")
                service = service['title']
                pricing = element.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                buy_options = {service:pricing}
                buy_list.append(buy_options)
                
        return buy_list

In [31]:
def get_detail(soup_object):
    labels  = []
    values = []
    table_info = soup_object.find("div", {"class":"title-info"})
    rows = table_info.find_all("div", {"class":"detail-infos"})
    for row in rows:
        label = row.find("div", {"class":"detail-infos__subheading"}).text.replace('\n',"")
        value = row.find("div", {"class":"detail-infos__value"}).text.replace('\n',"")
        labels.append(label)
        values.append(value)
        
    detail = {labels[i]: values[i] for i in range(len(labels))}
        
    return detail

In [21]:
def get_best(driver):
            driver.implicitly_wait(0.5)
            '''
            best = WebDriverWait(driver=driver, timeout=5).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="base"]/div[2]/div/div[2]/div[2]/div[3]/div/div[2]/div/div[1]/div[1]/div/div[1]'))
            )
            best.click()
            
            time.sleep(5)
            '''
            
            page_source =driver.page_source
            hun=BeautifulSoup(page_source,'html.parser')
            stream_best = get_stream_options(hun)
            buy_best = get_buy_options(hun)
            rent_best = get_rent_options(hun)
            
            options = {"stream_best": stream_best,
                    "rent_best":rent_best,
                    "buy_best":buy_best}
            
            return options
        
def get_free(driver, bts):
            time.sleep(5)
            bts[1].click()
            
            time.sleep(5)
            
            page_source =driver.page_source
            hun=BeautifulSoup(page_source,'html.parser')
            stream_free = get_stream_options(hun)
            buy_free = get_buy_options(hun)
            rent_free = get_rent_options(hun)
            
            
            options = {"stream_free": stream_free,
                    "rent_free":rent_free,
                    "buy_free":buy_free}
        
            return options
        
def get_sd(driver, bts):
            driver.implicitly_wait(0.5)
            bts[2].click()
            
            time.sleep(5)
            
            page_source =driver.page_source
            hun=BeautifulSoup(page_source,'html.parser')
            stream_sd = get_stream_options(hun)
            buy_sd = get_buy_options(hun)
            rent_sd = get_rent_options(hun)
            
            options = {"stream_sd": stream_sd,
                    "rent_sd":rent_sd,
                    "buy_sd":buy_sd}
        
            return options
        
def get_hd(driver, bts):
        driver.implicitly_wait(0.5)
            
        bts[3].click()
            
            
        time.sleep(5)
            
        page_source =driver.page_source
        
        hun=BeautifulSoup(page_source,'html.parser')
        stream_hd = get_stream_options(hun)
        buy_hd = get_buy_options(hun)
        rent_hd = get_rent_options(hun)
            
        options = {"stream_hd": stream_hd,
                    "rent_hd":rent_hd,
                    "buy_hd":buy_hd}
        
        return options
        
      
def get_4k(driver, bts):   
        driver.implicitly_wait(0.5)
        bts[4].click()
            
        time.sleep(5)
            
        page_source =  driver.page_source
        hun=BeautifulSoup(page_source,'html.parser')
        stream_4k = get_stream_options(hun)
        buy_4k = get_buy_options(hun)
        rent_4k = get_rent_options(hun)       
            
        options = {"stream_4k": stream_4k,
                    "rent_4k":rent_4k,
                    "buy_4k":buy_4k}
        
        return options

def get_season_links(driver, link):
        season_links = []
        driver.implicitly_wait(1)
        tvs = driver.find_elements(By.XPATH, '//*[@id="base"]/div[2]/div/div[2]/div[2]/div[2]/div[2]/div/div/div')
        tv_sns = WebDriverWait(driver=driver, timeout=5).until(
                        EC.presence_of_element_located((By.XPATH, '//*[@id="base"]/div[2]/div/div[2]/div[2]/div[2]/div[2]/div/div/div'))
                        )
        sel = Selector(text=driver.page_source)
        tvs = sel.xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div[2]/div[2]/div/div/div')
        for tv in tvs:
                tv_links = tv.css('::attr(href)').getall()
                for l in tv_links:
                        season_link  = base_url+l
                        sn_link = {"season_link":season_link,
                            "tv_link":link}
                        season_links.append(sn_link)
                
        return season_links

def get_synopsis(driver):     
    synopsis = driver.find_element(By.XPATH, '//*[@id="base"]/div[2]/div/div[2]/div[2]/div[2]/div[2]/div[2]/p').text
    return synopsis

link = "https://www.justwatch.com/us/tv-show/stargirl"

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(link)
banner = WebDriverWait(driver=driver, timeout=1).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app"]/div[5]/div[2]/div[2]/button[1]'))
        )
                
banner.click()
time.sleep(2)
bts = driver.find_elements(By.CLASS_NAME,'jw-chip-button')
try:
    imdb_link = WebDriverWait(driver=driver, timeout=1).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="base"]/div[2]/div/div[1]/div/aside/div[1]/div[3]/div[1]/div[2]/div/div[2]/a'))
        )
               
    imdb_link = imdb_link.get_attribute('href')
except:
        imdb_link = None
                
page_source =  driver.page_source
                
hun=BeautifulSoup(page_source,'html.parser')

In [32]:
def get_data(link):
            data = []
            driver = webdriver.Chrome(ChromeDriverManager().install())
            with driver as driver:
                driver.get(link)
                
                banner = WebDriverWait(driver=driver, timeout=1).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="app"]/div[5]/div[2]/div[2]/button[1]'))
                    )
                
                banner.click()
                
                time.sleep(2)
                bts = driver.find_elements(By.CLASS_NAME,'jw-chip-button')
                
                try:
                    imdb_link = WebDriverWait(driver=driver, timeout=1).until(
                        EC.presence_of_element_located((By.XPATH, '//*[@id="base"]/div[2]/div/div[1]/div/aside/div[1]/div[3]/div[1]/div[2]/div/div[2]/a'))
                    )
               
                    imdb_link = imdb_link.get_attribute('href')
                except:
                    imdb_link = None
                
                page_source =  driver.page_source
                
                hun=BeautifulSoup(page_source,'html.parser')
                
                
                try:
                    detail = get_detail(hun)
                except:
                    detail = None
                    
                try:
                    title_name = hun.find("div", {"class":"title-block"})
                    title_name = title_name.find("h1").text.replace('\n',"")
                except:
                    title_name = None
                    
                try:
                    season_1_year = hun.find("span", {"class":"text-muted"}).text.replace('\n',"")
                except:
                    year = None
                
                try:
                    prices_best = get_best(driver)
                except:
                    prices_best = None
                
                try:
                    prices_free = get_free(driver, bts)
                except:
                    prices_free = None
                    
                try:
                    prices_sd = get_sd(driver, bts)
                except:
                    prices_sd = None
                    
                try:
                    prices_hd = get_hd(driver, bts)
                except:
                    prices_hd = None
                    
                try:   
                    prices_4k = get_4k(driver, bts)
                except:
                    prices_4k = None

                try:
                    season_urls = get_season_links(driver, link)
                except:
                    season_urls = None
        
                title_url = link
                
                try:
                    now = datetime.now()
                    date_scraped = now.strftime("%d/%m/%Y %H:%M:%S")
                except:
                    date_scraped = None
                
                
                tv = {"title":title_name, "season_1_year":season_1_year, "season_urls":season_urls,"detail":detail, "fourkprices":prices_4k, "hdprices":prices_hd, "sdprices":prices_sd, "free":prices_free, "bestprices":prices_best,"imdblink":imdb_link,"title_link": title_url}
                
                return  tv
        
        
        

In [58]:
for link in title_urls[20:30]:
    data = []
    data.append(get_data(link))
    df = pd.DataFrame.from_dict(data).astype("string")
    df.to_sql('data_table', engine, if_exists='append', index=False)
    sql = "UPDATE tv_links SET is_scraped = 1 WHERE title_link = (%s)"
    titlelink = (link,)
    mycursor.execute(sql, titlelink)
    justwatch_tv_db.commit()

C:\Users\user\AppData\Local\Temp\ipykernel_5076\922501315.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
C:\Users\user\AppData\Local\Temp\ipykernel_5076\922501315.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
C:\Users\user\AppData\Local\Temp\ipykernel_5076\922501315.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
C:\Users\user\AppData\Local\Temp\ipykernel_5076\922501315.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
C:\Users\user\AppData\Local\Temp\ipykernel_5076\922501315.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Servic

In [48]:
title_urls

In [29]:
df = pd.read_sql('SELECT * FROM data_table', con=justwatch_tv_db)

c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [30]:
df

,title,season_1_year,season_urls,detail,fourkprices,hdprices,sdprices,free,bestprices,imdblink,title_link
0,Snowpiercer,(2020),[{'season_link': 'https://www.justwatch.com/us...,None,"{'stream_4k': [], 'rent_4k': [], 'buy_4k': []}","{'stream_hd': [{'DIRECTV': '3 Seasons HD '}, {...","{'stream_sd': [{'TNT': '3 Seasons '}, {'Spectr...",{'stream_free': [{'Spectrum On Demand': '3 Sea...,"{'stream_best': [{'DIRECTV': '3 Seasons HD '},...",https://www.imdb.com/title/tt6156584/?ref_=ref...,https://www.justwatch.com/us/tv-show/snowpiercer
1,Snowpiercer,(2020),[{'season_link': 'https://www.justwatch.com/us...,None,"{'stream_4k': [], 'rent_4k': [], 'buy_4k': []}","{'stream_hd': [{'DIRECTV': '3 Seasons HD '}, {...","{'stream_sd': [{'TNT': '3 Seasons '}, {'Spectr...",{'stream_free': [{'Spectrum On Demand': '3 Sea...,"{'stream_best': [{'DIRECTV': '3 Seasons HD '},...",https://www.imdb.com/title/tt6156584/?ref_=ref...,https://www.justwatch.com/us/tv-show/snowpiercer
2,DC's Stargirl,(2020),None,None,None,None,None,None,"{'stream_best': [{'HBO Max': '2 Seasons HD '},...",https://www.imdb.com/title/tt8722888/?ref_=ref...,https://www.justwatch.com/us/tv-show/stargirl
3,Snowpiercer,(2020),[{'season_link': 'https://www.justwatch.com/us...,None,"{'stream_4k': [], 'rent_4k': [], 'buy_4k': []}","{'stream_hd': [{'DIRECTV': '3 Seasons HD '}, {...","{'stream_sd': [{'TNT': '3 Seasons '}, {'Spectr...",{'stream_free': [{'Spectrum On Demand': '3 Sea...,"{'stream_best': [{'DIRECTV': '3 Seasons HD '},...",https://www.imdb.com/title/tt6156584/?ref_=ref...,https://www.justwatch.com/us/tv-show/snowpiercer
4,DC's Stargirl,(2020),None,None,None,None,None,None,"{'stream_best': [{'HBO Max': '2 Seasons HD '},...",https://www.imdb.com/title/tt8722888/?ref_=ref...,https://www.justwatch.com/us/tv-show/stargirl
5,Star Trek: Picard,(2020),[{'season_link': 'https://www.justwatch.com/us...,None,"{'stream_4k': [], 'rent_4k': [], 'buy_4k': []}",{'stream_hd': [{'Paramount Plus': '2 Seasons H...,{'stream_sd': [{'Paramount Plus': '2 Seasons '...,{'stream_free': [{'Paramount Plus': '1 Season ...,{'stream_best': [{'Paramount Plus': '2 Seasons...,https://www.imdb.com/title/tt8806524/?ref_=ref...,https://www.justwatch.com/us/tv-show/star-trek...
6,Snowpiercer,(2020),[{'season_link': 'https://www.justwatch.com/us...,None,"{'stream_4k': [], 'rent_4k': [], 'buy_4k': []}","{'stream_hd': [{'DIRECTV': '3 Seasons HD '}, {...","{'stream_sd': [{'TNT': '3 Seasons '}, {'Spectr...",{'stream_free': [{'Spectrum On Demand': '3 Sea...,"{'stream_best': [{'DIRECTV': '3 Seasons HD '},...",https://www.imdb.com/title/tt6156584/?ref_=ref...,https://www.justwatch.com/us/tv-show/snowpiercer
7,DC's Stargirl,(2020),None,None,None,None,None,None,"{'stream_best': [{'HBO Max': '2 Seasons HD '},...",https://www.imdb.com/title/tt8722888/?ref_=ref...,https://www.justwatch.com/us/tv-show/stargirl
8,Star Trek: Picard,(2020),[{'season_link': 'https://www.justwatch.com/us...,None,"{'stream_4k': [], 'rent_4k': [], 'buy_4k': []}",{'stream_hd': [{'Paramount Plus': '2 Seasons H...,{'stream_sd': [{'Paramount Plus': '2 Seasons '...,{'stream_free': [{'Paramount Plus': '1 Season ...,{'stream_best': [{'Paramount Plus': '2 Seasons...,https://www.imdb.com/title/tt8806524/?ref_=ref...,https://www.justwatch.com/us/tv-show/star-trek...
9,Close Enough,(2020),[{'season_link': 'https://www.justwatch.com/us...,None,"{'stream_4k': [], 'rent_4k': [], 'buy_4k': []}","{'stream_hd': [], 'rent_hd': [], 'buy_hd': [{'...","{'stream_sd': [], 'rent_sd': [], 'buy_sd': [{'...","{'stream_free': [], 'rent_free': [], 'buy_free...","{'stream_best': [], 'rent_best': [], 'buy_best...",https://www.imdb.com/title/tt6994156/?ref_=ref...,https://www.justwatch.com/us/tv-show/close-enough


In [ ]:
k_df = df['fourkprices'].apply(pd.Series)
h_df = df['hdprices'].apply(pd.Series)
s_df = df['sdprices'].apply(pd.Series)
f_df = df['free'].apply(pd.Series)
b_df = df['bestprices'].apply(pd.Series)
d_df = df['detail'].apply(pd.Series)
d_df


In [ ]:
#df['season_urls']
season_links_df = df['season_urls'].apply(pd.Series)
df_sns = pd.json_normalize(season_links_df[0])
df_sns = pd.concat([df_sns, df['season_1_year'].reindex(df.index)], axis=1)
df_sns.to_sql('justwatch_season_urls', engine, if_exists='replace', index=False)

In [ ]:
df = pd.concat([df, k_df.reindex(df.index)], axis=1)
df = pd.concat([df, h_df.reindex(df.index)], axis=1)
df = pd.concat([df, s_df.reindex(df.index)], axis=1)
df = pd.concat([df, f_df.reindex(df.index)], axis=1)
df = pd.concat([df, b_df.reindex(df.index)], axis=1)
df = pd.concat([df, d_df.reindex(df.index)], axis=1)

df_c = df.drop(['4k prices', 'hd prices', 'sd prices', 'free', 'best prices', 'detail', 'season_1_year'], axis=1)

In [ ]:
df_c = df_c.applymap(str)

In [ ]:
df_c.to_sql('justwatch_data_clean', engine, if_exists='replace', index=False)

In [ ]:
df_c.to_csv("tv.csv")